MovieLensを利用してサンプルを作成する  
MovieLensのダウンロード

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import sys
import os
sys.path.append(os.path.dirname(Path().resolve()))
from datasets.movie_lens import MovieLensDataset

In [2]:
movielens_datasets = MovieLensDataset()

In [3]:
movielens_datasets.ratings.head()

,userId,movieId,rating,timestamp,userIdx,movieIdx
0,1,307,3.5,1256677221,0,304
1,6,307,4.0,832059248,5,304
2,56,307,4.0,1383625728,55,304
3,71,307,5.0,1257795414,70,304
4,84,307,3.0,999055519,83,304


# TensorFlowを使う

In [4]:
!pip install tensorflow


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
import tensorflow as tf
from collections import Counter
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.8/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: cannot open shared object file: No such file or directory']
  warnings.warn(f"file system 

In [6]:
def create_model(user_num: int, item_num: int, dimension: int = 10) -> tf.keras.models.Model:
    user_input_layer = tf.keras.layers.Input(shape=(1, ))
    item_input_layer = tf.keras.layers.Input(shape=(1, ))
    user_embedding_layer = tf.keras.layers.Embedding(user_num, dimension)(user_input_layer)
    item_embedding_layer = tf.keras.layers.Embedding(item_num, dimension)(item_input_layer)
    user_bias_layer = tf.keras.layers.Embedding(user_num, 1)(user_input_layer)
    item_bias_layer = tf.keras.layers.Embedding(item_num, 1)(item_input_layer)

    x = tf.keras.layers.Dot(axes=2)([user_embedding_layer, item_embedding_layer])
    x = tf.keras.layers.Add()([x, user_bias_layer, item_bias_layer])
    x = tf.keras.layers.Flatten()(x)

    model = tf.keras.models.Model(inputs=[user_input_layer, item_input_layer], outputs=x)
    opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)

    model.compile(
        loss=tf.keras.losses.MeanSquaredError(),
        optimizer=opt,
        metrics=[tf.keras.metrics.RootMeanSquaredError()],
    )

    return model

In [7]:
# 学習データと検証データの用意
train, test = train_test_split(movielens_datasets.ratings, test_size=0.3, shuffle=True, random_state=42)
print(train.shape, test.shape)

(19427410, 6) (8326034, 6)


In [8]:
# モデルの作成
user_num = len(movielens_datasets.users)
content_num = len(movielens_datasets.movies)
model = create_model(user_num, content_num)

In [9]:
# 学習
tf.keras.utils.plot_model(model, show_shapes=True)

# callback関数
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=0
    )
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'keras.h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True,
    mode='min',
    verbose=0
)

# 学習実行
result = model.fit(
    x=[train['userIdx'].values, train['movieIdx'].values],
    y=train['rating'].values,
    epochs=200,
    batch_size=1024,
    validation_data=(
        [test['userIdx'].values, test['movieIdx'].values],
        test['rating'].values
    ),
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Epoch 1/200
18973/18973 [==============================] - 82s 4ms/step - loss: 2.0103 - root_mean_squared_error: 1.4178 - val_loss: 1.2443 - val_root_mean_squared_error: 1.1155
Epoch 2/200
18973/18973 [==============================] - 83s 4ms/step - loss: 1.1343 - root_mean_squared_error: 1.0650 - val_loss: 1.0667 - val_root_mean_squared_error: 1.0328
Epoch 3/200
18973/18973 [==============================] - 84s 4ms/step - loss: 1.0175 - root_mean_squared_error: 1.0087 - val_loss: 0.9910 - val_root_mean_squared_error: 0.9955
Epoch 4/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.9584 - root_mean_squared_error: 0.9790 - val_loss: 0.9470 - val_root_mean_squared_error: 0.9731
Epoch 5/200
18973/18973 [==============================] - 86s 5ms/step - loss: 0.9205 - root_mean_squared_error: 0.9594 - val_loss: 0.9159

Epoch 46/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5396 - root_mean_squared_error: 0.7346 - val_loss: 0.6639 - val_root_mean_squared_error: 0.8148
Epoch 47/200
18973/18973 [==============================] - 83s 4ms/step - loss: 0.5383 - root_mean_squared_error: 0.7337 - val_loss: 0.6636 - val_root_mean_squared_error: 0.8146
Epoch 48/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5372 - root_mean_squared_error: 0.7329 - val_loss: 0.6635 - val_root_mean_squared_error: 0.8145
Epoch 49/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5360 - root_mean_squared_error: 0.7321 - val_loss: 0.6634 - val_root_mean_squared_error: 0.8145
Epoch 50/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5349 - root_mean_squared_error: 0.7314 - val_loss: 0.6633 - val_root_mean_squared_error: 0.8144
Epoch 51/200
18973/18973 [==============================] - 84s 4ms/step - loss: 0.5339 - root_mean_squar

In [10]:
result

In [11]:
model.save('../datasets/matrix-factorization.h5')

In [29]:
model.predict([np.array([283214, 283214, 283214]), np.array([1, 2, 3])], verbose=1)

1/1 [==============================] - 0s 36ms/step


array([[3.952248 ],
       [3.679933 ],
       [3.3663383]], dtype=float32)